<a href="https://colab.research.google.com/github/samsas556/ELE400Pitot_Omer/blob/main/Lab2_ELE767.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn . model_selection import KFold

mode = 3 #0= perceptron, 1=MLP a 2 couche , 2= MLP a 2 couche avec abandon , 3= MLP a 3 couche, 4=modele au choix

validation_mystere = 0 # verifie les donne mystere avec la configuration du mode choisis

epoque = 15


def training( model , data_ds , epochs =10 , n_fold =5 , verbose_train =1 , verbose_test=1) :
  """
  TRAINING
  Fonction servant a entrainer un model et en verifiant les capacites du modele
  sur les donnees de test
  @Arguments :
    model { tensorflow . python . keras . engine . sequential . Sequential } : Modele
    tensorflow a entrainer
    data_ds {pd. DataFrame } : Base de donnees d’ entrainement
    epochs { int } : Nombre d’epoques a faire pour l’ entrainement du model
    n_fold { int } : Nombre de fold pour le kfold
    verbose_train { int } : 1 pour afficher l’avancement de l’apprentissage ,
    0 aucun affichage
    verbose_test { int} : 1 pour afficher l’avancement de l’evaluation ,
    0 aucun affichage
    @Return :
    { dict } : ’ train_accuracy ’ { list } : liste de la precision du modele sur la
    base de donnees d’ entrainement
    ’train_loss ’ { list } : liste des resultat de la fonction de
    perte du modele sur la base de donnees d’ entrainement
    ’ test_accuracy ’ { list } : liste de la precision du modele sur
    la base de donnees de test
    ’test_loss ’ { list } : liste des resultats de la fonction de
    perte du modele sur la base de donnees de test
  """
  # Initialisation des listes recevants
  train_accuracy = []
  train_loss = []
  test_accuracy = []
  test_loss = []

  # Creer les bases de donnees pour l’ entrainement
  train_set , test_set = create_sets ( data_ds , n_fold )

  # Evaluer le modele avant l’ entrainement
  result = model . evaluate ( train_set , verbose = verbose_train )
  train_accuracy . append ( result [1])
  train_loss . append ( result [0])
  # ---
  result = model . evaluate ( test_set , verbose = verbose_test )
  test_accuracy . append ( result [1])
  test_loss . append ( result [0])

  # Boucle d’ entrainement

  for e in range(epochs):
    print ('Epoch #{} '.format(e +1))
    # ---
    result = model.fit(train_set , epochs=1 , verbose = verbose_train)
    train_accuracy.append( result.history['accuracy'][-1])
    train_loss.append(result.history['loss'][-1])
    # ---
    result = model.evaluate( test_set , verbose = verbose_test )
    test_accuracy.append( result[1])
    test_loss.append( result[0])

  return {'train_accuracy': train_accuracy ,
          'train_loss': train_loss ,
          'test_accuracy': test_accuracy ,
          'test_loss': test_loss }

def figure( training_dict , suptitle , figsize =(12 ,6) ):
  """
  FIGURE
  Produit une figure avec les statistiques de la fonction ’training ’
  @Arguments :
  training_dict { dict ( list )} : Dictionnaire retournee par la fonction
  training
  figsize { Tuple } : Dimension de la figure
  """
  global mode

  # Demarre une figure avec la taille (en pouce ) donner en arguments
  plt.figure(figsize = figsize , facecolor ='w')
  # Titre global de la figure
  plt.suptitle( suptitle )
  # Premiere sous - figure -> Precision du modele durant l’ entrainement
  plt.subplot(121)
  plt.title('Precision du modele ')
  plt.xlabel('Nombre d\' epoques ')
  plt.ylabel('Precision de la classification ')
  # Produit une courbe pour chaque base de donnees
  plt.plot(training_dict['train_accuracy'], 'r', label ='training')
  plt.plot(training_dict['test_accuracy'], 'y', label ='test')
  # Limite de la precision pour rendre les valeurs entre 0% et 100% visibles
  plt.ylim(0 ,1.01)
  # Produire un grille en arriere - plan
  #plt.grid(b=True , which ='major', color = '#666666 ' , linestyle = '-')
  plt.grid()
  plt.minorticks_on()
  #plt.grid(b= True , which ='minor ', color ='#999999 ', linestyle ='-', alpha =0.2)
  plt.legend() # Afficher les labels des courbes
  # Deuxieme sous - figure -> Resultat de la fonction de perte durant l’entrainement
  plt.subplot(122)
  plt.title('Perte du modele ')
  plt.xlabel('Nombre d\' epoques ')
  plt.ylabel('Resultat de la fonction de perte')
  # Produire une courbe pour chaque base de donnees
  plt.plot(training_dict['train_loss'],'r', label ='training')
  plt.plot(training_dict['test_loss'],'y', label ='test');
  # S’assurer que la valeur minimum de l’ordonnee soit 0
  plt.ylim(0);
  # Produire un grille en arriere - plan
  #plt.grid(b=True , which ='major', color = '#666666 ' , linestyle =  '-')
  plt.grid()
  plt.minorticks_on()
  #plt.grid(b= True , which ='minor', color ='#999999 ', linestyle ='-', alpha =0.2)
  plt.legend()

  if mode==0:
    plt.savefig("graph_perceptron.png")
  elif mode ==1:
    plt.savefig("graph_MLP2c.png")
  elif mode ==2:
    plt.savefig("graph_MLP2dropout.png")
  elif mode ==3:
    plt.savefig("graph_MLP3c.png")
  return

def create_sets(data_ds, n_fold =5):
  """
  CREATE_SETS
    Transforme les bases de donnees afin d’etre compatible avec l’ entrainement
    du modele avec la methode " fit "
  @Arguments :
    data_ds {pd. DataFrame } : Base de donnees d’ entrainement
    n_fold { int } : nombre de division pour le kfold
  @Return :
    { tensorflow.BatchDataset } : Base de donnees d’ entrainement
    { tensorflow.BatchDataset } : Base de donnees de test
  """
  # Melanger les valeurs de la base de donnees
  data = data_ds.sample(frac=1).reset_index(drop=True)

  # Diviser les variables et les resultats
  data_output = data.pop("number")

  # Extraire les indexes pour le kfold une seule fois
  for train_index , test_index in KFold( n_fold ).split( data ) :
    # Creer la base de donnees d’ entrainement
    X_train = (data.values)[train_index]
    y_train = (data_output.values )[train_index]
    ### Creer la base de donnees de test
    X_test = (data.values) [test_index]
    y_test = (data_output.values)[test_index]
    break

  # Transformer les bases de donnees pour etre utilisees directement dans fit
  # Les donnees sont melangees une autre fois avant d’etre retournees
  train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
  train_set = train_set.shuffle(len(X_train))
  train_set = train_set.batch(1)

  test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))
  test_set = test_set.shuffle(len(X_test))
  test_set = test_set.batch(1)

  return train_set , test_set


#============================main code =============================#


data_set = pd.read_csv("data.csv")


if validation_mystere ==1:

  _mystere_ds = pd.read_csv("mystere.csv")
  print(_mystere_ds.info())
  mystere_ds = tf.data. Dataset.from_tensor_slices(_mystere_ds.values)
  mystere_set = mystere_ds.batch(1)

  result = []

  for model_dir in ['modeles/perceptron','modeles/mlp2','modeles/mlp2Dropout','modeles/mlp3']:
    model = tf.keras.models.load_model(model_dir)
    prediction = model.predict(mystere_set)
    # Retourne l’index ayant la plus grande valeur
    result.append( np.argmax(prediction , axis =1))


  # Transferer la liste en array de numpy
  result = np.array(result)
  # Afficher les resultats
  print(" Resultat de tous les modeles ")
  print(result)
  print(" Resultat commun entre les modeles ")
  print((stats.mode(result,axis =0)) [0])

elif validation_mystere ==0:

  if mode==0:
    # Creer un modele sequentiel
    perceptron = tf.keras.Sequential([layers.Dense(4 , activation='sigmoid', dtype ='float64')])
    # Compiler le modele pour l’ entrainement
    perceptron.compile(optimizer ='SGD',loss = tf.keras.losses.SparseCategoricalCrossentropy() ,metrics =['accuracy'])

    result_perceptron = training(perceptron,data_set,epochs=epoque)
    figure( result_perceptron , " Apprentissage d’un perceptron pour la classification des signaux sonores ")

    perceptron.save('modeles/perceptron')

  elif mode==1:

    mlp2c = tf.keras.Sequential([
    layers.Dense(512 , activation ='sigmoid', dtype ='float64'),
    layers.Dense(10 , activation ='sigmoid', dtype ='float64')])

    # Compiler le modele pour l’ entrainement
    mlp2c.compile( optimizer ='SGD',loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics =['accuracy'])

    result = training(mlp2c,data_set,epochs=epoque)
    figure(result , " Apprentissage d’un MLP a 2 couche pour la classification des signaux sonores ")

    mlp2c.save('modeles/mlp2')

  elif mode==2:

    mlp2c = tf.keras.Sequential([
    layers.Dense(512 , activation ='sigmoid', dtype ='float64'),
    layers.Dense(10 , activation ='sigmoid', dtype ='float64')])
    layers.Dropout(0.2 , dtype ='float64')

    # Compiler le modele pour l’ entrainement
    mlp2c.compile( optimizer ='SGD',loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics =['accuracy'])

    result = training(mlp2c,data_set,epochs=epoque)
    figure(result , " Apprentissage d’un MLP a 2 couche avec abandon pour la classification des signaux sonores ")

    mlp2c.save('modeles/mlp2Dropout')

  elif mode==3:

    mlp3c = tf.keras.Sequential([
            layers.Dense(512 , activation ='relu', dtype ='float64') ,
            layers.Dense(128 , activation ='relu', dtype ='float64') ,
            layers.Dense(10 , activation ='relu', dtype ='float64') ,
            layers.Softmax(dtype ='float64')])

    # Compiler le modele pour l’ entrainement
    mlp3c.compile( optimizer ='SGD',loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics =['accuracy'])

    result = training(mlp3c,data_set,epochs=epoque)
    figure(result , " Apprentissage d’un MLP a 2 couche avec abandon pour la classification des signaux sonores ")

    mlp3c.save('modeles/mlp3')

  elif mode==4:

    model_choix = tf.keras.Sequential([
            layers.Dense(512 , activation ='relu', dtype ='float64') ,
            layers.Dense(128 , activation ='relu', dtype ='float64') ,
            layers.Dense(10 , activation ='relu', dtype ='float64') ,
            layers.Softmax(dtype ='float64')])

    # Compiler le modele pour l’ entrainement
    model_choix.compile( optimizer ='SGD',loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics =['accuracy'])

    result = training(model_choix,data_set,epochs=epoque)
    figure(result , " Apprentissage d’un MLP a 2 couche avec abandon pour la classification des signaux sonores ")

    model_choix.save('modeles/model_choix')






662/712 [==========================>...] - ETA: 0s - loss: 2.3715 - accuracy: 0.1178

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')